In [1]:
import os
import pandas as pd
from textblob import *
import nltk
import numpy as np
import openpyxl
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from textblob import *
'''
    WELFAKE
'''



data = pd.read_csv(r"../data/Des_fake_news/WELFAKE_PROCESSED.csv")
data["flag"] = data["label"]
data = data.sample(5000)
data



,Unnamed: 0.1,Unnamed: 0,title,text,label,text_tb_pol,text_tb_sub,title_tb_pol,title_tb_sub,title_vader_comp,title_vader_neg,title_vader_neu,title_vader_pos,text_vader_neg,text_vader_neu,text_vader_pos,flag
2425,52523,52523,OBAMA IGNORES PLANNED PARENTHOOD BABY PARTS HA...,Justice the Obama way Instead of investigating...,1,0.013468,0.405724,0.000000,0.000000,-0.4824,0.228,0.772,0.000,0.044,0.851,0.104,1
2290,17817,17817,Cop Caught on His Own Body Camera Stealing Mon...,Home Badge Abuse Cop Caught on His Own Bod...,1,0.007261,0.402271,0.262500,0.525000,-0.8176,0.459,0.541,0.000,0.162,0.797,0.041,1
853,22879,22879,Congress s fight over Iran deal enters new phase,The Obama administration says new visa rules p...,0,0.003346,0.411703,0.136364,0.454545,-0.3818,0.245,0.755,0.000,0.121,0.800,0.079,0
2517,59352,59352,Spains Industry Minister Steps Down Over Panam...,MADRID The Spanish minister of industry e...,0,0.092785,0.288384,-0.009596,0.371717,0.0000,0.000,1.000,0.000,0.088,0.846,0.066,0
2577,54521,54521,Almost Missing After Boat Carrying Migrants S...,Get short URL As many as people are missi...,1,0.100000,0.306061,-0.150000,0.025000,-0.2280,0.175,0.825,0.000,0.075,0.889,0.035,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4066,33675,33675,HAMMER TIME TRUMP POINTS OUT THE LIES BEHIND ...,This ad is spot on,1,0.000000,0.000000,0.333333,0.766667,0.7954,0.130,0.516,0.354,0.000,1.000,0.000,1
3484,42202,42202,PELOSI LOSES IT Calls Tax Cuts End of the Wor...,Nancy Pelosi just got even more embarrassing t...,1,0.189286,0.502619,-0.300000,0.100000,-0.6408,0.368,0.632,0.000,0.052,0.791,0.157,1
2019,29210,29210,DEMOCRATS CAUGHT Paying Halfway House Patients...,A lawless party whose end always justifies the...,1,0.000000,0.000000,0.000000,0.000000,0.0000,0.000,1.000,0.000,0.102,0.753,0.144,1
2147,4713,4713,Magnates Twin Goals Fighting Climate Change a...,MONTAUK N Y On a sunny morning late thi...,0,0.130400,0.446699,0.136364,0.454545,-0.3612,0.161,0.839,0.000,0.062,0.800,0.138,0


In [2]:
'''
    Add tf-idf vectorizer
'''

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords
import pandas as pd

def tf_idf_vectorize(df: pd.DataFrame, corpus: pd.Series, vocabulary: list[str]) -> tuple[list[str], pd.DataFrame]:
    stop_words = stopwords.words("english")
    vectorizer = TfidfVectorizer(strip_accents="ascii", lowercase=True, stop_words=stop_words, max_features=500, ngram_range=(1,3), vocabulary=vocabulary)
    features = vectorizer.fit_transform(corpus).toarray()
    names = vectorizer.get_feature_names_out()
    headers = [f"__word{i}" for i in range(len(names))]
    feature_frame = pd.DataFrame(features, columns=headers)
    #final = pd.concat([df, feature_frame], axis=1)
    return (names, feature_frame)


def get_full_vocabulary(corpus: pd.Series) -> list[str]:
    vectorizer = TfidfVectorizer(strip_accents="ascii", lowercase=True, stop_words=stopwords.words("english"), max_features=500, ngram_range=(1,3))
    vectorizer.fit_transform(corpus).toarray()
    return vectorizer.get_feature_names_out()


def add_tf_idf_vector(x: pd.DataFrame, partial_corpus: pd.Series, full_corpus: pd.Series) -> pd.DataFrame:
    temp = pd.concat((x, partial_corpus), axis=1).reset_index()
    # first, we need a vocabulary from the entire dataset
    vocabulary = get_full_vocabulary(full_corpus)

    tf_idf_vector = tf_idf_vectorize(temp, temp[partial_corpus.name], vocabulary=vocabulary)[1].reset_index()
    temp = temp.reset_index()
    temp = pd.concat((temp, tf_idf_vector), axis=1).drop([partial_corpus.name, "index"], axis=1)
    return temp



In [3]:
from openpyxl import load_workbook, Workbook

'''
    Define filtered dataset, classes, features, dataframe for model accuracies, and excel file for results
'''

'''Drop NA'''
filtered = data.dropna()
classes = filtered["flag"]

filtered["corpus"] = filtered.apply(lambda x: " ".join([x["title"], x["text"]]), axis=1)


# raw polarity and subjectivity scores from Textblob, Vader
features = filtered[["text_tb_pol",	"text_tb_sub",	"title_tb_pol",	"title_tb_sub",	"title_vader_comp",	"title_vader_neg",	
              "title_vader_neu",	"title_vader_pos",	"text_vader_neg",	"text_vader_neu",	"text_vader_pos"]]

# dataframe to store accuracies for NN and log regression
#accuracy_df = pd.DataFrame(columns=["polarity",
#               "tfidf",
#               "combined"])


EXCEL_FILE = r"../data/Des_fake_news/Sentiment_Analysis_Results/WELFAKE_RESULTS.xlsx"
# overwrite book if exists
book = Workbook()
book.save(filename=EXCEL_FILE)
book.close()


In [4]:
'''
    POLARITY + TF-IDF
'''
from sklearn.discriminant_analysis import StandardScaler
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.svm import SVC
from sklearn.model_selection import KFold
import openpyxl.drawing
from sklearn import metrics
import matplotlib.pyplot as plt
import io



# average accuracies
comb_output = {"train" : None, "test" : None}
comb_matrices = []


comb_fpr = []
comb_tpr = []
comb_thresh = []

comb_auc = 0


train_total = 0
test_total = 0
run_count = 0


#lbgfs
x = features
y = classes
folds = 5
kf = KFold(n_splits=folds)

for i, (train_index, test_index) in enumerate(kf.split(x)):
    print(i)
    scaler = StandardScaler()
    print("tf_idf training")
    x_train = add_tf_idf_vector(x.iloc[train_index], filtered["corpus"].iloc[train_index], filtered["corpus"])
    x_train = scaler.fit_transform(x_train)

    print("tf_idf testing")
    x_test = add_tf_idf_vector(x.iloc[test_index], filtered["corpus"].iloc[test_index], filtered["corpus"])
    x_test = scaler.transform(x_test)
    y_train = y.iloc[train_index]
    y_test = y.iloc[test_index]

    comb_ensemble = VotingClassifier(estimators=[
    ('lr', LogisticRegression(max_iter=500)),
    ('rf', RandomForestClassifier(n_estimators=100, random_state=42)),
    ('svc', SVC(kernel='rbf', probability=True, random_state=42))
    ], voting='soft')
    print("ensemble fitting")
    comb_ensemble.fit(x_train, y_train)
    y_pred = comb_ensemble.predict(x_test)

    fpr, tpr, thresh = metrics.roc_curve(y_test, y_pred, pos_label=1)
    comb_fpr.append(fpr)
    comb_tpr.append(tpr)
    comb_thresh.append(thresh)
    comb_auc += metrics.auc(fpr, tpr)

    comb_matrices.append(confusion_matrix(y_test, y_pred))
    test_total += accuracy_score(y_test, y_pred)

    y_train_pred = comb_ensemble.predict(x_train)
    train_total += accuracy_score(y_train, y_train_pred)

    run_count += 1

print(f"Train accuracy on {folds}-fold cross-validation: {train_total / float(run_count)}")
print(f"Test accuracy on {folds}-fold cross-validation: {test_total / float(run_count)}")

comb_output["test"] = [(test_total / run_count)]
comb_output["train"] = [(train_total / run_count)]
comb_output = pd.DataFrame(comb_output)

print("Average confusion matrix:")
comb_matrices = np.mean(np.array(comb_matrices), axis=0)
print(comb_matrices)


comb_fpr = np.mean(np.array(comb_fpr), axis=0)
comb_tpr = np.mean(np.array(comb_tpr), axis=0)
comb_thresh = np.mean(np.array(comb_thresh), axis=0)

comb_auc = comb_auc / float(run_count)










comb_output

0
tf_idf training
tf_idf testing
ensemble fitting
1
tf_idf training
tf_idf testing
ensemble fitting
2
tf_idf training
tf_idf testing
ensemble fitting
3
tf_idf training
tf_idf testing
ensemble fitting
4
tf_idf training
tf_idf testing
ensemble fitting
Train accuracy on 5-fold cross-validation: 0.9952500000000001
Test accuracy on 5-fold cross-validation: 0.9184000000000001
Average confusion matrix:
[[441.4  44.4]
 [ 37.2 477. ]]


,train,test
0,0.99525,0.9184


In [5]:
'''
    POLARITY
'''
from sklearn.discriminant_analysis import StandardScaler
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.svm import SVC
from sklearn.model_selection import KFold


# average accuracies
pol_output = {"train" : None, "test" : None}
pol_matrices = []


pol_fpr = []
pol_tpr = []
pol_thresh = []

pol_auc = 0

train_total = 0
test_total = 0
run_count = 0


#lbgfs
x = features
y = classes
folds = 5
kf = KFold(n_splits=folds)

for i, (train_index, test_index) in enumerate(kf.split(x)):
    print(i)
    scaler = StandardScaler()
  
    x_train = x.iloc[train_index]
    x_train = scaler.fit_transform(x_train)

    x_test = x.iloc[test_index]
    x_test = scaler.transform(x_test)
    y_train = y.iloc[train_index]
    y_test = y.iloc[test_index]

    pol_ensemble = VotingClassifier(estimators=[
    ('lr', LogisticRegression(max_iter=500)),
    ('rf', RandomForestClassifier(n_estimators=100, random_state=42)),
    ('svc', SVC(kernel='rbf', probability=True, random_state=42))
    ], voting='soft')
    print("ensemble fitting")
    pol_ensemble.fit(x_train, y_train)
    y_pred = pol_ensemble.predict(x_test)
    fpr, tpr, thresh = metrics.roc_curve(y_test, y_pred, pos_label=1)
    pol_fpr.append(fpr)
    pol_tpr.append(tpr)
    pol_thresh.append(thresh)
    pol_auc += metrics.auc(fpr, tpr)

    pol_matrices.append(confusion_matrix(y_test, y_pred))
    test_total += accuracy_score(y_test, y_pred)

    y_train_pred = pol_ensemble.predict(x_train)
    train_total += accuracy_score(y_train, y_train_pred)

    run_count += 1

print(f"Train accuracy on {folds}-fold cross-validation: {train_total / float(run_count)}")
print(f"Test accuracy on {folds}-fold cross-validation: {test_total / float(run_count)}")

pol_output["test"] = [(test_total / run_count)]
pol_output["train"] = [(train_total / run_count)]
pol_output = pd.DataFrame(pol_output)

print("Average confusion matrix:")
pol_matrices = np.mean(np.array(pol_matrices), axis=0)
print(pol_matrices)

pol_fpr = np.mean(np.array(pol_fpr), axis=0)
pol_tpr = np.mean(np.array(pol_tpr), axis=0)
pol_thresh = np.mean(np.array(pol_thresh), axis=0)

pol_auc = pol_auc / float(run_count)




pol_output

0
ensemble fitting
1
ensemble fitting
2
ensemble fitting
3
ensemble fitting
4
ensemble fitting
Train accuracy on 5-fold cross-validation: 0.88695
Test accuracy on 5-fold cross-validation: 0.7023999999999999
Average confusion matrix:
[[341.4 144.4]
 [153.2 361. ]]


,train,test
0,0.88695,0.7024


In [6]:
'''
    TF-IDF
'''
from sklearn.discriminant_analysis import StandardScaler
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.svm import SVC
from sklearn.model_selection import KFold


# average accuracies
tf_output = {"train" : None, "test" : None}
tf_matrices = []

train_total = 0
test_total = 0
run_count = 0

tf_fpr = []
tf_tpr = []
tf_thresh = []

tf_auc = 0

#lbgfs
x = features
y = classes
folds = 5
kf = KFold(n_splits=folds)

for i, (train_index, test_index) in enumerate(kf.split(x)):
    print(i)
    scaler = StandardScaler()
    print("tf_idf training")
    x_train = add_tf_idf_vector(x.iloc[train_index], filtered["corpus"].iloc[train_index], filtered["corpus"]).drop(features, axis=1)
    x_train = scaler.fit_transform(x_train)

    print("tf_idf testing")
    x_test = add_tf_idf_vector(x.iloc[test_index], filtered["corpus"].iloc[test_index], filtered["corpus"]).drop(features, axis=1)
    x_test = scaler.transform(x_test)
    y_train = y.iloc[train_index]
    y_test = y.iloc[test_index]

    tf_ensemble = VotingClassifier(estimators=[
    ('lr', LogisticRegression(max_iter=500)),
    ('rf', RandomForestClassifier(n_estimators=100, random_state=42)),
    ('svc', SVC(kernel='rbf', probability=True, random_state=42))
    ], voting='soft')
    print("ensemble fitting")
    tf_ensemble.fit(x_train, y_train)
    y_pred = tf_ensemble.predict(x_test)
    fpr, tpr, thresh = metrics.roc_curve(y_test, y_pred, pos_label=1)
    tf_fpr.append(fpr)
    tf_tpr.append(tpr)
    tf_thresh.append(thresh)
    tf_auc += metrics.auc(fpr, tpr)


    tf_matrices.append(confusion_matrix(y_test, y_pred))
    test_total += accuracy_score(y_test, y_pred)

    y_train_pred = tf_ensemble.predict(x_train)
    train_total += accuracy_score(y_train, y_train_pred)

    run_count += 1

print(f"Train accuracy on {folds}-fold cross-validation: {train_total / float(run_count)}")
print(f"Test accuracy on {folds}-fold cross-validation: {test_total / float(run_count)}")

tf_output["test"] = [(test_total / run_count)]
tf_output["train"] = [(train_total / run_count)]
tf_output = pd.DataFrame(tf_output)

print("Average confusion matrix:")
tf_matrices = np.mean(np.array(tf_matrices), axis=0)
print(tf_matrices)

tf_fpr = np.mean(np.array(tf_fpr), axis=0)
tf_tpr = np.mean(np.array(tf_tpr), axis=0)
tf_thresh = np.mean(np.array(tf_thresh), axis=0)

tf_auc = tf_auc / float(run_count)



tf_output

0
tf_idf training
tf_idf testing
ensemble fitting
1
tf_idf training
tf_idf testing
ensemble fitting
2
tf_idf training
tf_idf testing
ensemble fitting
3
tf_idf training
tf_idf testing
ensemble fitting
4
tf_idf training
tf_idf testing
ensemble fitting
Train accuracy on 5-fold cross-validation: 0.99445
Test accuracy on 5-fold cross-validation: 0.9196
Average confusion matrix:
[[441.4  44.4]
 [ 36.  478.2]]


,train,test
0,0.99445,0.9196


In [7]:
'''
    combine accuracies into one table
'''

final_results = pd.concat((pol_output, tf_output, comb_output), axis=0)
final_results.index = ["Polarity", "Tf-Idf", "Polarity + Tf-Idf"]
final_results

,train,test
Polarity,0.88695,0.7024
Tf-Idf,0.99445,0.9196
Polarity + Tf-Idf,0.99525,0.9184


In [8]:
'''
    Finally, save accuracy metrics to the spreadsheet
'''




book = load_workbook(EXCEL_FILE)
writer = pd.ExcelWriter(EXCEL_FILE, engine="openpyxl")
writer.book = book
final_results.to_excel(writer, sheet_name=f"accuracies")
book.save(filename=EXCEL_FILE)
book.close()



C:\Users\desmo\AppData\Local\Temp\ipykernel_8452\3534906820.py:10: FutureWarning: Setting the `book` attribute is not part of the public API, usage can give unexpected or corrupted results and will be removed in a future version
  writer.book = book


In [9]:
import openpyxl.drawing
from sklearn import metrics
import matplotlib.pyplot as plt
import io


plt.figure()
lw = 2
plt.plot(pol_fpr, pol_tpr,
 lw=lw, label='Polarity: (%0.2f)' % pol_auc)
plt.plot(tf_fpr, tf_tpr,
 lw=lw, label='Tf-Idf: (%0.2f)' % tf_auc)
plt.plot(comb_fpr, comb_tpr,
 lw=lw, label='Polarity + Tf-Idf: (%0.2f)' % comb_auc)
plt.plot([0, 1], [0, 1], color='black', lw=lw, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic (ROC)')
plt.legend(loc="lower right")
#plt.show()


# save figure as PNG
png = io.BytesIO()
plt.savefig(png, format="png")


# write PNG to excel file
book = load_workbook(EXCEL_FILE)
ws = book.active

img = openpyxl.drawing.image.Image(png)
img.anchor = "A1"
ws.add_image(img)
book.save(filename=EXCEL_FILE)
plt.close()
book.close()

